In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wg
import pylab as pyl

d2r = np.pi/180

versorx = np.asarray([1,0,0])
versory = np.asarray([0,1,0])
versorz = np.asarray([0,0,1])


def transf(vec,Rot_y=45,Rot_z=-10):
    # transforma un vector [x,y,z] en el sistema de referencia que usa
    # matplotlib (con ejes y,z en la pantalla de la compu y el x saliendo para afuera)
    # al mismo sistema pero rotado Rot_y alrededor del eje y y luego Rot_z alrededor
    # del eje z
    vec=np.asarray(vec)
    x,y,z=vec
    Rot_y*=d2r
    Rot_z*=d2r
    ny = x*np.sin(Rot_z) + y*np.cos(Rot_z)
    nz = -x*np.sin(Rot_y)*np.cos(Rot_z) + y*np.sin(Rot_y)*np.sin(Rot_z) + z*np.cos(Rot_y)
    return(ny,nz)


def f(M,e=0):
    # calcula la anomalia verdadera
    if np.shape(M) == ():
        E_old = 0
        E_new = 100
        while abs(E_old-E_new)>1e-4:
            E_oldnew = E_new
            E_new = M + e*np.sin(E_old)
            E_old = E_oldnew
        E = E_new
        f = 2.*np.arctan(np.sqrt((1+e)/(1-e))*np.tan(E/2.))
    else:
        N = len(M)
        f = []
        for Mi in M:
            E_old = 0
            E_new = 100
            while abs(E_old-E_new)>1e-4:
                E_oldnew = E_new
                E_new = Mi + e*np.sin(E_old)
                E_old = E_oldnew
            E = E_new
            f.append(2.*np.arctan(np.sqrt((1+e)/(1-e))*np.tan(E/2.)))
    return(f)


def orbit(f_ang='all',e=0,w=0,Omega=0,inc=0,a=1):
    # devuelve posiciones x,y,z de una orbita dados
    # los parametros orbitales (a=1)
    w*=d2r
    Omega*=d2r
    inc*=d2r
    if e<0.9: nM = 360*2
    else: nM = 360*5
    if type(f_ang)==str:
        Mdom=np.linspace(0,2*np.pi,nM)
        ff = np.asarray(f(Mdom,e))
    else: 
        ff=f_ang*d2r
    x = np.cos(Omega)*np.cos(w+ff) - np.sin(Omega)*np.sin(w+ff)*np.cos(inc)
    y = np.sin(Omega)*np.cos(w+ff) + np.cos(Omega)*np.sin(w+ff)*np.cos(inc)
    z = np.sin(w+ff)*np.sin(inc)
    
    r = a*(1-e**2)/(1+e*np.cos(ff))
    
    return([r*x,r*y,r*z])


def sep_orbit(orb):
    # separa una lista de puntos [xl,yl,zl] que describan una orbita
    # en segmentos que cruzan encima del plano de referencia y debajo
    # en particular queremos usar trazos distintas para tales segmentos
    sign_old = np.sign(orb[2][0])
    orbx_=[]
    orby_=[] # nuevas orbitas segmentadas
    orbz_=[]
    orbits=[]
    for i in range(len(orb[2])):
        sign_new=np.sign(orb[2][i])
        if sign_new==sign_old:
            orbx_.append(orb[0][i])
            orby_.append(orb[1][i])
            orbz_.append(orb[2][i])
        else:
            orbits.append([orbx_,orby_,orbz_])
            
            orbx_=[]
            orby_=[] # nuevas orbitas segmentadas
            orbz_=[]
            
            
            orbx_.append(orb[0][i])
            orby_.append(orb[1][i])
            orbz_.append(orb[2][i])
            
            sign_old=sign_new
    orbits.append([orbx_,orby_,orbz_])
    return(orbits)

def angles_plot(Rot_y,Rot_z,e,w,Omega,inc,show_ang):
    tk = dict(Rot_y=Rot_y,Rot_z=Rot_z) # transform kwargs
    ok = dict(e=e,w=w,Omega=Omega,inc=inc)  # orbit kwargs
    
    plt.figure(figsize=(8,8),dpi=120)
    
    # orbita
    if e<0.99 and inc!=0 and inc!=180:
        orbit_segs = sep_orbit(orbit(**ok))  # segmentamos la orbita para que tenga un estilo de linea
                                             # distinto arriba que debajo del plano de referencia
        for orbit_segi in orbit_segs:
            orbit_segi_p = transf(orbit_segi,**tk)
            if np.sign(orbit_segi[2][0])==1 or inc==0:   # segmento arriba del plano
                plt.plot(orbit_segi_p[0],orbit_segi_p[1],lw=.75,alpha=.5,c='k')
            else:   # segmento debajo del plano
                plt.plot(orbit_segi_p[0],orbit_segi_p[1],lw=.5,alpha=.5,c='k',linestyle='dotted')   
    else: # si la orbita es super excentrica no hace falta separar porque no habra casi nada debajo del plano
        orbit_ = orbit(**ok)
        orbit_p= transf(orbit_,**tk)
        plt.plot(orbit_p[0],orbit_p[1],lw=.75,alpha=.5,c='k')
    
    
    #### lineas horizontales de los ejes
    ax_x = [[-1,1],  [0,0], [0,0]]  # van del -1 al 1
    ax_y = [ [0,0], [-1,1], [0,0]]
    
    ax_x_p = transf(ax_x,**tk)
    ax_y_p = transf(ax_y,**tk)

    plt.plot(ax_x_p[0], ax_x_p[1],  c='k',linestyle='dashed',lw=.5,alpha=.75,zorder=10)
    plt.plot(ax_y_p[0], ax_y_p[1],  c='k',linestyle='dashed',lw=.5,alpha=.75,zorder=10)
    
    
    #### direccion de referencia
    g_point = [1+0.1,0,0]
    g_point_p = transf(g_point,**tk)
    plt.text(g_point_p[0],g_point_p[1],r'$\gamma$',fontsize=12)
    
    
    #### circulo unitario
    dom    = np.linspace(0,2*np.pi,360*2)
    circ   = [np.cos(dom),np.sin(dom),[0]*360*2]
    circ_p = transf(circ,**tk) # circulo proyectado
    plt.plot(circ_p[0],circ_p[1],lw=0.5,alpha=0.75,c='k',linestyle='dashed')
    
    
    #### linea del nodo ascendente
    if inc!=0 and inc!=180:
        aps_vec = [[np.cos(Omega*d2r)],[np.sin(Omega*d2r)],[0]]  # del origen hasta la interseccion orbita - plano de referencia
                                                                 # notemos que es solo la mitad de la linea
        
        # quiero que se corte cuando choca con la orbita, pero el ancho de la orbita
        # cambia con w
        r1 = (1-e**2)/(1+e*np.cos(w*d2r))
        r2 = (1-e**2)/(1+e*np.cos(w*d2r + np.pi))  # r1(f += 180°)
        
        aps_vec1 = r1*np.asarray(aps_vec)
        aps_vec2 = r2*np.asarray(aps_vec)
        
        aps_vec=np.zeros((3,2))
        aps_vec[0]=[-aps_vec2[0][0],aps_vec1[0][0]]
        aps_vec[1]=[-aps_vec2[1][0],aps_vec1[1][0]]
        
        aps_vec_p = transf(aps_vec,**tk)
        
        plt.plot(aps_vec_p[0],aps_vec_p[1],lw=.5,alpha=.75,c='k')
        
    
    #### linea del pericentro
    if e!=0:
        peri = orbit(f_ang=0,**ok)
        afe  = orbit(f_ang=180,**ok)
        linea_peri = [[peri[0],afe[0]],[peri[1],afe[1]],[peri[2],afe[2]]]
        linea_peri = np.asarray(linea_peri)
        linea_peri_p = transf(linea_peri,**tk)
        plt.plot(linea_peri_p[0],linea_peri_p[1],lw=.5,alpha=.75,c='k')
        
        
    #### angulo de inc
    if show_ang and inc!=0 and inc!=180:
        Mdom=np.linspace(0,inc,360)
        circ_inc = orbit(f_ang=Mdom,Omega=Omega+90,w=0,inc=90,a=0.2)
        circ_inc_p=transf(circ_inc,**tk)

        plt.plot(circ_inc_p[0],circ_inc_p[1],lw=0.5,alpha=.75,c='k')   # arco de la cuña
        plt.plot([0,circ_inc_p[0][0]],[0,circ_inc_p[1][0]],lw=0.5,alpha=.75,c='k')   # una vara de la cuña
        plt.plot([0,circ_inc_p[0][-1]],[0,circ_inc_p[1][-1]],lw=0.5,alpha=.75,c='k')  # otra vara de la cuña
        
        # texto I
        inc_text_pos = orbit(f_ang=Mdom[180],Omega=Omega+90,w=0,inc=90,a=0.2+5*0.01)
        inc_text_pos_p = transf(inc_text_pos,**tk)
        plt.text(inc_text_pos_p[0],inc_text_pos_p[1],r'$\mathcal{I}$',fontsize=12)
        
        
    #### angulo de Omega
    if show_ang and inc!=0 and inc!=180 and Omega!=0 and Omega!=360:
        Mdom=np.linspace(0,Omega,360)
        circ_Om = orbit(f_ang=Mdom,a=0.275)
        circ_Om_p=transf(circ_Om,**tk)
        
        plt.plot(circ_Om_p[0],circ_Om_p[1],lw=0.5,alpha=.75,c='k')   # arco de la cuña
        
        # texto Omega
        Om_text_pos = orbit(f_ang=Mdom[180],a=0.3)
        Om_text_pos_p=transf(Om_text_pos,**tk)
        plt.text(Om_text_pos_p[0],Om_text_pos_p[1],r'$\mathcal{\Omega}$',fontsize=12)
        
    
    #### angulo de w
    if show_ang and e!=0 and w!=0 and inc!=0 and inc!=180:
        Mdom=np.linspace(0,w,360)
        circ_w = orbit(f_ang=Mdom,a=0.2,Omega=Omega,inc=inc)
        circ_w_p=transf(circ_w,**tk)
        
        plt.plot(circ_w_p[0],circ_w_p[1],lw=0.5,alpha=.75,c='k')
        
        # texto w
        w_text_pos = orbit(f_ang=Mdom[180],a=0.2,Omega=Omega,inc=inc)
        w_text_pos_p=transf(w_text_pos,**tk)
        plt.text(w_text_pos_p[0],w_text_pos_p[1],r'$\mathcal{\omega}$',fontsize=12)
        
        
    #### flechas de versores
    arrx_dy = transf(versorx/5,Rot_y=Rot_y,Rot_z=Rot_z)[0]
    arrx_dz = transf(versorx/5,Rot_y=Rot_y,Rot_z=Rot_z)[1]
    pyl.arrow( 0., 0., arrx_dy, arrx_dz, fc="k", ec="k",head_width=0.035, head_length=0.07,zorder=110)

    arry_dy = transf(versory/5,Rot_y=Rot_y,Rot_z=Rot_z)[0]
    arry_dz = transf(versory/5,Rot_y=Rot_y,Rot_z=Rot_z)[1]
    pyl.arrow( 0., 0., arry_dy, arry_dz, fc="k", ec="k",head_width=0.035, head_length=0.07,zorder=110)

    arrz_dy = transf(versorz/5,Rot_y=Rot_y,Rot_z=Rot_z)[0]
    arrz_dz = transf(versorz/5,Rot_y=Rot_y,Rot_z=Rot_z)[1]
    pyl.arrow( 0., 0., arrz_dy, arrz_dz, fc="k", ec="k",head_width=0.035, head_length=0.07,zorder=110)

    #### ultimos parametros de estilo de plot
    plt.gca().axis('equal')
    plt.gca().axis('off')
    
    plt.xlim(-1.5,1.5)
    plt.ylim(-1.5,1.5)
    plt.show()


wg.interact(angles_plot, Rot_y=(0,90), Rot_z=(-90,90), e=(0,.99), w=(0,360), Omega=(0,360), inc=(0,360),show_ang=False)

interactive(children=(IntSlider(value=45, description='Rot_y', max=90), IntSlider(value=0, description='Rot_z'…

<function __main__.angles_plot(Rot_y, Rot_z, e, w, Omega, inc, show_ang)>